In [57]:
import shutil
import requests
import pandas as pd

from enum import Enum
from os import path, mkdir
from decimal import Decimal
from datetime import datetime
from pydantic import ValidationError
from pydantic.dataclasses import dataclass

In [58]:
class Timeframe(Enum):
    HOURLY = "1h"
    DAILY = "1d"
    WEEKLY = "1w"
    MONTHLY = "1M"

@dataclass
class Candle:
    time: datetime
    open: Decimal
    high: Decimal
    low: Decimal
    close: Decimal
    volume: Decimal

@dataclass
class Ticker:
    symbol: str
    price: Decimal
    volume: Decimal

def highest_volume(tickers: list[Ticker]) -> list[Ticker]:
    """Sorts tickers based on volume and removes tickers with no volume."""
    filtered = filter(lambda x: x.price > 0 and x.volume > 0, tickers)
    tickers = sorted(filtered, key=lambda x: x.price * x.volume, reverse=True)
    return tickers

def filter_symbols(market: str, tickers: list[Ticker], blacklist: list[str] = []) -> list[Ticker]:
    """Returns list of tickers denominated in the provided market, excluding symbols that are blacklisted."""
    filtered = filter(lambda x: x.symbol.endswith(market) and not blacklisted(x.symbol, blacklist), tickers)
    return list(filtered)

def blacklisted(symbol: str, blacklist: list[str]) -> bool:
    """Checks if a blacklisted symbol is part of a given market and returns True if it is."""
    if len(blacklist) > 0:
        for item in blacklist:
            if symbol.startswith(item) or symbol.endswith(item):
                return True
    return False

class Binance():
    BASEURL = "https://api.binance.com"
    ENDPOINTS = {
        "ticker": "/api/v3/ticker/24hr",
        "price": "/api/v3/ticker/price",
        "kline": "/api/v3/klines"
    }

    def __init__(self) -> None:
        pass

    def markets(self) -> list[Ticker]:
        url = Binance.BASEURL + Binance.ENDPOINTS["ticker"]
        r = requests.get(url)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        return list(map(lambda x: Ticker(x["symbol"], x["lastPrice"], x["volume"]), r.json()))

    def kline(self, symbol: str, interval: Timeframe) -> list[Candle]:
        url = Binance.BASEURL + Binance.ENDPOINTS["kline"]
        payload = { "symbol": symbol, "interval": interval.value }
        r = requests.get(url, params=payload)

        if r.status_code != 200:
            raise requests.exceptions.HTTPError(r.json())

        klines = []
        for kline in r.json():
            candle = Candle(*kline[:6])
            klines.append(candle)

        return klines

**Data Selection**

In [59]:
binance = Binance()
markets = binance.markets()

blacklist = ["UPUSDT", "DOWNUSDT", "BEARUSDT", "BULLUSDT"]
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS"]

usdt_markets = filter_symbols("USDT", markets, blacklist + stablecoins)
usdt_volume = highest_volume(usdt_markets)

**Data Download**

Could be a lot faster with async and without sleep, but since downloading the data is not time critical it's more important to make sure to not hit the rate limits.

In [60]:
tf = Timeframe.HOURLY
download = usdt_volume[:100]

print(f"Downloading {len(download)} USDT markets...")
usdt_data = {}
for ticker in download:
    usdt_data[ticker.symbol] = binance.kline(ticker.symbol, tf)
print("Finished downloading.")

Finished downloading.


In [61]:
import plotly.express as px

def relative(symbol, df, benchmark) -> pd.DataFrame:
    rel = df.reindex(benchmark.index, method="bfill")
    rel[symbol] = rel["close"].div(benchmark["close"] * rel["close"][0]) * benchmark["close"][0]
    return rel

def calc_relative(benchmark: str, data: dict[str, list[Candle]]) -> pd.DataFrame:
    bm = pd.DataFrame.from_dict(data[benchmark]).set_index("time")
    rel = pd.DataFrame(index = bm.index)
    for symbol, candles in data.items():
        try:
            df = pd.DataFrame.from_dict(candles).set_index("time")
            df = relative(symbol, df, bm)
            rel.insert(len(rel.columns), symbol, df[symbol])
        except TypeError:
            print(f"Error while processing {symbol}")
    return rel

In [65]:
day = {k: v[-24:] for k, v in usdt_data.items()}
week = {k: v[(-24*7):] for k, v in usdt_data.items()}
rel_day = calc_relative("BTCUSDT", day)
rel_week = calc_relative("BTCUSDT", week)

diff = (rel_day < 1.05) & (rel_day > 0.95)
rel_day = rel_day.drop(columns=rel_day.columns[diff.all()])

diff = (rel_week < 1.15) & (rel_week > 0.85)
rel_week = rel_week.drop(columns=rel_week.columns[diff.all()])

fig_l = px.line(rel_day, x = rel_day.index, y = rel_day.columns)
fig_r = px.line(rel_week, x = rel_week.index, y = rel_week.columns)

fig_l.show()
fig_r.show()

Error while processing COCOSUSDT
Error while processing NEBLUSDT
Error while processing AUDUSDT
Error while processing COCOSUSDT
Error while processing NEBLUSDT
Error while processing AUDUSDT


**Data Persistence**

Execute to save the downloaded kline data into individual csv-files for each downloaded market.

In [63]:
data_path = path.join("..", "data")

if path.exists(data_path):
    shutil.rmtree(data_path)

mkdir(data_path)

for symbol, data in usdt_data.items():
    df = pd.DataFrame.from_dict(data).set_index("time")
    df.to_csv(path.join(data_path, f"{symbol}_{tf.name.lower()}.csv"))

# Remove data from memory
# df = None
# usdt_data = None